In [1]:
# coding: utf-8
# наше всё
import numpy as np
import pandas as pd

# pandas settings
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

# for graphics
import seaborn as sns
import matplotlib.pyplot as plt

# for own libs
import sys
sys.path.append('//')

# lib for working with SQL
import data_load as dl

from pylab import rcParams
rcParams['figure.figsize'] = 12, 9

In [2]:
dl.get_data("""

SELECT distinct table_catalog, table_schema, table_name
FROM information_schema.columns 
where table_schema='sport'
limit 50

""")

,table_catalog,table_schema,table_name
0,postgres,sport,footbal_data1
1,postgres,sport,footbal_data2


In [4]:
%%time
# all files from github
root='data/football-data/data/results/'
import os
file_lst=[]
for path, subdirs, files in os.walk(root):
    for name in files:
        if ('.csv' in name)==True:
            file_lst.append(os.path.join(path, name))
            
# all in one dataframe
df_tot=pd.DataFrame()
for file in file_lst:
#     print(file)
    b=pd.read_csv(file)
    b['file']=file
    df_tot=pd.concat([df_tot, b])
df_tot['match_id']=range(len(df_tot))
df_tot['match_id']='fdr_'+df_tot['match_id'].astype('str')

CPU times: user 1min 4s, sys: 43.3 s, total: 1min 47s
Wall time: 1min 47s


In [5]:
len(df_tot)

1078214

In [6]:
df_tot['source'] = 'file:_'+df_tot['file']

columns_to_sql = [
       'match_id',
       'home', 'away', 'date', 'gh', 'ga', 'full_time', 
       'level', 'competition',
       'source'
            ]

df_to_sql = df_tot[columns_to_sql]

In [7]:
df_to_sql.sample(3)

,match_id,home,away,date,gh,ga,full_time,level,competition,source
1328,fdr_923990,Eepco,Mugher Cement,2011-04-06,2,2,F,national,ethiopia,file:_data/football-data/data/results/ethiopia.csv
6603,fdr_194402,Sport Huancayo,Fbc Melgar,2017-08-12,2,0,F,national,peru,file:_data/football-data/data/results/peru.csv
523,fdr_813282,African Lyon FC,Jkt Ruvu Stars,2010-09-20,1,2,F,national,tanzania,file:_data/football-data/data/results/tanzania.csv


In [8]:
len(df_to_sql)

1078214

In [9]:
main_engine=dl.get_engine('config.yaml')

In [10]:
def inser_data(df_to_sql, schema, table_name):
    import datetime
    now = datetime.datetime.now()
    df_to_sql['insert_time'] = now
    df_to_sql.to_sql(
                            table_name, 
                            con=main_engine, 
                            schema=schema,
                            if_exists='append', 
                            index=False
                        )
    print(str(len(df_to_sql))+' rows inserted to  '+table_name)

In [25]:
import math
import datetime
total_rows = len(df_to_sql)
batch_size = 100
num_of_batch = math.ceil(total_rows/batch_size)

print('There will be '+str(num_of_batch)+' insert interations')
for i in range(num_of_batch):
    now = datetime.datetime.now()
    share=np.round(100*i/num_of_batch,2)
    print(str(now)+' : '+str(share)+'%')
    inser_data(df_to_sql[i*batch_size:(i+1)*batch_size], 'sport', 'footbal_data')

There will be 10783 insert interations
2022-04-02 17:28:09.818705 : 0.0%
100 rows inserted to  footbal_data
2022-04-02 17:28:09.869934 : 0.01%
100 rows inserted to  footbal_data
2022-04-02 17:28:09.900394 : 0.02%
100 rows inserted to  footbal_data
2022-04-02 17:28:09.925790 : 0.03%
100 rows inserted to  footbal_data
2022-04-02 17:28:09.951092 : 0.04%
100 rows inserted to  footbal_data
2022-04-02 17:28:09.978547 : 0.05%
100 rows inserted to  footbal_data
2022-04-02 17:28:10.011963 : 0.06%


<ipython-input-10-f0f55254b1d7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_sql['insert_time'] = now


100 rows inserted to  footbal_data
2022-04-02 17:28:10.041282 : 0.06%
100 rows inserted to  footbal_data
2022-04-02 17:28:10.070266 : 0.07%
100 rows inserted to  footbal_data
2022-04-02 17:28:10.098062 : 0.08%
100 rows inserted to  footbal_data
2022-04-02 17:28:10.123463 : 0.09%
100 rows inserted to  footbal_data
2022-04-02 17:28:10.149899 : 0.1%
100 rows inserted to  footbal_data
2022-04-02 17:28:10.182376 : 0.11%
100 rows inserted to  footbal_data
2022-04-02 17:28:10.205053 : 0.12%
100 rows inserted to  footbal_data
2022-04-02 17:28:10.234942 : 0.13%
100 rows inserted to  footbal_data
2022-04-02 17:28:10.262866 : 0.14%
100 rows inserted to  footbal_data
2022-04-02 17:28:10.291557 : 0.15%
100 rows inserted to  footbal_data
2022-04-02 17:28:10.318392 : 0.16%
100 rows inserted to  footbal_data
2022-04-02 17:28:10.345385 : 0.17%
100 rows inserted to  footbal_data
2022-04-02 17:28:10.391528 : 0.18%
100 rows inserted to  footbal_data
2022-04-02 17:28:10.420625 : 0.19%
100 rows inserted to 

100 rows inserted to  footbal_data
2022-04-02 17:28:13.530998 : 1.22%
100 rows inserted to  footbal_data
2022-04-02 17:28:13.553753 : 1.23%
100 rows inserted to  footbal_data
2022-04-02 17:28:13.577951 : 1.24%
100 rows inserted to  footbal_data
2022-04-02 17:28:13.600772 : 1.25%
100 rows inserted to  footbal_data
2022-04-02 17:28:13.617914 : 1.26%
100 rows inserted to  footbal_data
2022-04-02 17:28:13.635258 : 1.27%
100 rows inserted to  footbal_data
2022-04-02 17:28:13.652716 : 1.28%
100 rows inserted to  footbal_data
2022-04-02 17:28:13.670428 : 1.29%
100 rows inserted to  footbal_data
2022-04-02 17:28:13.687137 : 1.3%
100 rows inserted to  footbal_data
2022-04-02 17:28:13.703411 : 1.31%
100 rows inserted to  footbal_data
2022-04-02 17:28:13.719819 : 1.32%
100 rows inserted to  footbal_data
2022-04-02 17:28:13.738170 : 1.33%
100 rows inserted to  footbal_data
2022-04-02 17:28:13.755800 : 1.34%
100 rows inserted to  footbal_data
2022-04-02 17:28:13.774945 : 1.34%
100 rows inserted to 

100 rows inserted to  footbal_data
2022-04-02 17:28:16.362958 : 2.35%
100 rows inserted to  footbal_data
2022-04-02 17:28:16.388074 : 2.36%
100 rows inserted to  footbal_data
2022-04-02 17:28:16.406550 : 2.36%
100 rows inserted to  footbal_data
2022-04-02 17:28:16.428886 : 2.37%
100 rows inserted to  footbal_data
2022-04-02 17:28:16.450266 : 2.38%
100 rows inserted to  footbal_data
2022-04-02 17:28:16.471731 : 2.39%
100 rows inserted to  footbal_data
2022-04-02 17:28:16.488593 : 2.4%
100 rows inserted to  footbal_data
2022-04-02 17:28:16.506428 : 2.41%
100 rows inserted to  footbal_data
2022-04-02 17:28:16.525139 : 2.42%
100 rows inserted to  footbal_data
2022-04-02 17:28:16.550312 : 2.43%
100 rows inserted to  footbal_data
2022-04-02 17:28:16.575140 : 2.44%
100 rows inserted to  footbal_data
2022-04-02 17:28:16.597180 : 2.45%
100 rows inserted to  footbal_data
2022-04-02 17:28:16.634597 : 2.46%
100 rows inserted to  footbal_data
2022-04-02 17:28:16.655691 : 2.47%
100 rows inserted to 

...

100 rows inserted to  footbal_data
2022-04-02 17:32:31.526730 : 94.84%
100 rows inserted to  footbal_data
2022-04-02 17:32:31.549926 : 94.85%
100 rows inserted to  footbal_data
2022-04-02 17:32:31.589902 : 94.86%
100 rows inserted to  footbal_data
2022-04-02 17:32:31.650975 : 94.87%
100 rows inserted to  footbal_data
2022-04-02 17:32:31.693075 : 94.88%
100 rows inserted to  footbal_data
2022-04-02 17:32:31.718229 : 94.89%
100 rows inserted to  footbal_data
2022-04-02 17:32:31.753281 : 94.9%
100 rows inserted to  footbal_data
2022-04-02 17:32:31.786411 : 94.91%
100 rows inserted to  footbal_data
2022-04-02 17:32:31.812524 : 94.92%
100 rows inserted to  footbal_data
2022-04-02 17:32:31.841634 : 94.93%
100 rows inserted to  footbal_data
2022-04-02 17:32:31.868686 : 94.94%
100 rows inserted to  footbal_data
2022-04-02 17:32:31.893317 : 94.95%
100 rows inserted to  footbal_data
2022-04-02 17:32:31.915492 : 94.96%
100 rows inserted to  footbal_data
2022-04-02 17:32:31.934586 : 94.96%
100 row

100 rows inserted to  footbal_data
2022-04-02 17:32:34.600288 : 95.97%
100 rows inserted to  footbal_data
2022-04-02 17:32:34.622963 : 95.98%
100 rows inserted to  footbal_data
2022-04-02 17:32:34.646436 : 95.98%
100 rows inserted to  footbal_data
2022-04-02 17:32:34.669000 : 95.99%
100 rows inserted to  footbal_data
2022-04-02 17:32:34.692341 : 96.0%
100 rows inserted to  footbal_data
2022-04-02 17:32:34.715884 : 96.01%
100 rows inserted to  footbal_data
2022-04-02 17:32:34.737717 : 96.02%
100 rows inserted to  footbal_data
2022-04-02 17:32:34.760020 : 96.03%
100 rows inserted to  footbal_data
2022-04-02 17:32:34.785189 : 96.04%
100 rows inserted to  footbal_data
2022-04-02 17:32:34.811752 : 96.05%
100 rows inserted to  footbal_data
2022-04-02 17:32:34.834975 : 96.06%
100 rows inserted to  footbal_data
2022-04-02 17:32:34.862637 : 96.07%
100 rows inserted to  footbal_data
2022-04-02 17:32:34.887821 : 96.08%
100 rows inserted to  footbal_data
2022-04-02 17:32:34.910771 : 96.09%
100 row

100 rows inserted to  footbal_data
2022-04-02 17:32:37.629321 : 97.07%
100 rows inserted to  footbal_data
2022-04-02 17:32:37.661513 : 97.08%
100 rows inserted to  footbal_data
2022-04-02 17:32:37.690621 : 97.09%
100 rows inserted to  footbal_data
2022-04-02 17:32:37.716040 : 97.1%
100 rows inserted to  footbal_data
2022-04-02 17:32:37.748032 : 97.11%
100 rows inserted to  footbal_data
2022-04-02 17:32:37.779155 : 97.12%
100 rows inserted to  footbal_data
2022-04-02 17:32:37.799098 : 97.13%
100 rows inserted to  footbal_data
2022-04-02 17:32:37.817846 : 97.13%
100 rows inserted to  footbal_data
2022-04-02 17:32:37.837284 : 97.14%
100 rows inserted to  footbal_data
2022-04-02 17:32:37.858537 : 97.15%
100 rows inserted to  footbal_data
2022-04-02 17:32:37.880578 : 97.16%
100 rows inserted to  footbal_data
2022-04-02 17:32:37.899693 : 97.17%
100 rows inserted to  footbal_data
2022-04-02 17:32:37.922533 : 97.18%
100 rows inserted to  footbal_data
2022-04-02 17:32:37.946848 : 97.19%
100 row

100 rows inserted to  footbal_data
2022-04-02 17:32:40.879587 : 98.21%
100 rows inserted to  footbal_data
2022-04-02 17:32:40.902478 : 98.22%
100 rows inserted to  footbal_data
2022-04-02 17:32:40.920571 : 98.23%
100 rows inserted to  footbal_data
2022-04-02 17:32:40.949167 : 98.24%
100 rows inserted to  footbal_data
2022-04-02 17:32:40.987069 : 98.25%
100 rows inserted to  footbal_data
2022-04-02 17:32:41.020028 : 98.26%
100 rows inserted to  footbal_data
2022-04-02 17:32:41.041810 : 98.27%
100 rows inserted to  footbal_data
2022-04-02 17:32:41.063467 : 98.28%
100 rows inserted to  footbal_data
2022-04-02 17:32:41.087471 : 98.28%
100 rows inserted to  footbal_data
2022-04-02 17:32:41.110144 : 98.29%
100 rows inserted to  footbal_data
2022-04-02 17:32:41.138340 : 98.3%
100 rows inserted to  footbal_data
2022-04-02 17:32:41.158888 : 98.31%
100 rows inserted to  footbal_data
2022-04-02 17:32:41.187004 : 98.32%
100 rows inserted to  footbal_data
2022-04-02 17:32:41.207876 : 98.33%
100 row

100 rows inserted to  footbal_data
2022-04-02 17:32:43.758122 : 99.3%
100 rows inserted to  footbal_data
2022-04-02 17:32:43.779179 : 99.3%
100 rows inserted to  footbal_data
2022-04-02 17:32:43.796969 : 99.31%
100 rows inserted to  footbal_data
2022-04-02 17:32:43.814023 : 99.32%
100 rows inserted to  footbal_data
2022-04-02 17:32:43.832460 : 99.33%
100 rows inserted to  footbal_data
2022-04-02 17:32:43.852301 : 99.34%
100 rows inserted to  footbal_data
2022-04-02 17:32:43.870792 : 99.35%
100 rows inserted to  footbal_data
2022-04-02 17:32:43.887939 : 99.36%
100 rows inserted to  footbal_data
2022-04-02 17:32:43.906172 : 99.37%
100 rows inserted to  footbal_data
2022-04-02 17:32:43.928247 : 99.38%
100 rows inserted to  footbal_data
2022-04-02 17:32:43.954098 : 99.39%
100 rows inserted to  footbal_data
2022-04-02 17:32:43.978664 : 99.4%
100 rows inserted to  footbal_data
2022-04-02 17:32:44.001834 : 99.41%
100 rows inserted to  footbal_data
2022-04-02 17:32:44.024918 : 99.42%
100 rows 

In [26]:
dl.get_data("""

SELECT count(*)
FROM sport.footbal_data
--limit 5

""")

,count
0,1078214


In [ ]:
token = '44ce50aab7504f17ae93e674300009f6'
avilable_leagues_lst = [
    'WC',
    'CL',
    'BL1',
    'DED',
    'BSA',
    'PD',
    'FL1',
    'ELC',
    'PPL',
    'EC',
    'SA',
    'PL',
    'CLI'
]
import requests
competitions_url='http://api.football-data.org/v2/competitions'

headers = {
    'X-Auth-Token': token,
            }

params = ()

r = requests.get(competitions_url, 
                 params=params, headers=headers)

print(r.status_code)

l1=[]
l2=[]
l3=[]
for i in range(r.json()['count']):
    l1.append(r.json()['competitions'][i]['name'])
    l2.append(r.json()['competitions'][i]['id'])
    l3.append(r.json()['competitions'][i]['code'])
    
tournaments_table=pd.DataFrame([l1, l2, l3]).T
tournaments_table.columns=['Name', 'ID', 'Code']

tournaments_table = tournaments_table[tournaments_table['Code'].isin(avilable_leagues_lst)]

In [ ]:
# footbal stat without api
# http://wildstat.ru/p/50/ch/EUR_CC_1988_1989

# big and reach api
# https://footystats.org/download-stats-csv

# big tile
# https://www.jokecamp.com/blog/guide-to-football-and-soccer-data-and-apis/

In [ ]:
# man: https://www.football-data.org/documentation/quickstart
# all competitions
url='http://api.football-data.org/v2/competitions'
# one competition all seasons 
# spain
# url='http://api.football-data.org/v2/competitions/2014'
# list of matches (with filters)
# url='http://api.football-data.org/v2/competitions/2014/matches'
# match data
# url='http://api.football-data.org/v2/matches/331356'
# team data
# url='http://api.football-data.org/v2/teams/78'